#### Engenharia de dados
**Nome**: MÁRCIO DE LIMA LOSS PUGNAL  
**Código Aluno**: 4052025000363  
**Curso**: Pós CIÊNCIA DE DADOS E ANALYTICS  
**Dataset**: [Board-Games] [fonte: https://www.kaggle.com/datasets/andrewmvd/board-games ]

#### Importação das bibliotecas, coleta e carga de dados.

In [0]:
# Importação de bibliotecas

# Manipulação de dados
import pandas as pd
import requests
#from io import StringIO

# Spark para processamento distribuído
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import (
    col,                          # Seleção e manipulação de colunas
    monotonically_increasing_id,  # Gerar IDs únicos
    regexp_replace,               # Substituir padrões usando regex
    explode,                      # Expandir arrays em múltiplas linhas
    split                         # Dividir strings em arrays
)

# Visualização de dados
import matplotlib.pyplot as plt               # Criação de gráficos básicos em Python
from matplotlib.ticker import FuncFormatter   # Formatar valores nos eixos (ex.: porcentagem, moeda)
from matplotlib.ticker import MultipleLocator # Definir espaçamento fixo entre marcas dos eixos
import seaborn as sns                         # Para gráficos estatísticos mais elaborados e prontos

In [0]:
# Coleta e carga de dados

# URL do dataset no GitHub
url = "https://raw.githubusercontent.com/marciopugnal/mvp-machine-learning/refs/heads/main/bgg_dataset.csv"

# Ler direto com pandas
df_dados = pd.read_csv(url, delimiter=';')

# Converter para Spark DataFrame
df_tabuleiro = spark.createDataFrame(df_dados)

# Informações do dataset carregado
print("Dataset carregado com sucesso!")
print(f"Total de registros: {df_tabuleiro.count()}")
print(f"Total de colunas..: {len(df_tabuleiro.columns)}")

#### Análise exploratória inicial

In [0]:
# Visualizar os dados
display(df_tabuleiro.limit(5))

In [0]:
# Exibir tipos de dados
print("Tipos de dados:")
display(spark.createDataFrame(df_tabuleiro.dtypes, ["Atributo", "Tipo"]))

In [0]:
# Mostrar as primeiras 5 linhas    
df_tabuleiro.select("ID", "Year Published", "Rating Average", "Complexity Average", "Owned Users").show(5)

#### Pré-processamentos dos dados

In [0]:
# Converter colunas de double para int
df_tabuleiro = df_tabuleiro.withColumn("ID", F.coalesce(F.col("ID"), F.lit(0)).cast("int"))
df_tabuleiro = df_tabuleiro.withColumn("Year Published", F.coalesce(F.col("Year Published"), F.lit(0)).cast("int"))
df_tabuleiro = df_tabuleiro.withColumn("Owned Users", F.coalesce(F.col("Owned Users"), F.lit(0)).cast("int"))

# Substituir vírgulas por ponto em decimais
df_tabuleiro = df_tabuleiro.withColumn("Rating Average", regexp_replace(col("Rating Average"), ",", ".").cast("double"))
df_tabuleiro = df_tabuleiro.withColumn("Complexity Average", regexp_replace(col("Complexity Average"), ",", ".").cast("double"))


In [0]:
# Converter outras colunas para int
colunas_int = ["Min Players", "Max Players", "Play Time", "Min Age", "Users Rated", "BGG Rank"]

# Percorrer as colunas e converter para int
for coluna in colunas_int:
    df_tabuleiro = df_tabuleiro.withColumn(coluna, F.coalesce(F.col(coluna), F.lit(0)).cast("int"))

In [0]:
df_tabuleiro.select("ID", "Year Published", "Rating Average", "Complexity Average", "Owned Users").show(5)

In [0]:
# Verificar valores ausentes
dados = []
for c in df_tabuleiro.columns:
    ausentes = df_tabuleiro.filter(col(c).isNull()).count()
    presentes = df_tabuleiro.filter(col(c).isNotNull()).count()
    dados.append((c, presentes, ausentes))

# Cria DataFrame com os resultados
df_resumo = spark.createDataFrame(dados, ["Atributos", "Valores Presentes", "Valores Ausentes"])
df_resumo.show(truncate=False)

In [0]:
# TRATAMENTO DE MISSING (valores ausentes) ---

# Eliminando linhas que tenham algum valor missing.
df_tabuleiro = df_tabuleiro.dropna(how='any')

dados = []
for c in df_tabuleiro.columns:
    ausentes = df_tabuleiro.filter(col(c).isNull()).count()
    presentes = df_tabuleiro.filter(col(c).isNotNull()).count()
    dados.append((c, presentes, ausentes))

# Cria DataFrame com os resultados
df_resumo = spark.createDataFrame(dados, ["Atributos", "Valores Presentes", "Valores Ausentes"])
df_resumo.show(truncate=False)

In [0]:
# OBTER VALORES MÍNIMO, MÁXIMO E QTD DE VALORES ÚNICOS

# Colunas que não são numéricas
colunas_excluir = ['Name', 'Mechanics', 'Domains']

dados = []
for c in df_tabuleiro.columns:
    uniq_val = df_tabuleiro.select(F.countDistinct(c)).collect()[0][0]
    
    if c in colunas_excluir:
        # Para colunas não numéricas, não calcular min/max
        min_val, max_val = "-", "-"
    else:
        # Para colunas numéricas, calcular min e max
        min_val = df_tabuleiro.agg(F.min(c)).collect()[0][0]
        max_val = df_tabuleiro.agg(F.max(c)).collect()[0][0]
    
    dados.append((c, min_val, max_val, uniq_val))

# Criar DataFrame com os resultados
range_valores = spark.createDataFrame(
    dados, ["Atributos", "Mínimo", "Máximo", "Qtd Valores Únicos"]
)

range_valores.show(truncate=False)

#### Tabelas do Modelo Estrela  
_É uma estrutura de banco de dados otimizada para consultas analíticas e relatórios, em contraste com os modelos normalizados (OLTP) otimizados para transações. O seu nome vem da sua forma visual, que se assemelha a uma estrela: uma tabela central (fatos) cercada por várias tabelas auxiliares (dimensões)._  
_Chave Substituta (Surrogate Key) é uma chave artificial, numérica e sequencial criada especificamente para o data warehouse, que substitui as chaves naturais (business keys) dos sistemas operacionais. Criada apenas para conectar tabelas de forma eficiente no data warehouse._

#### Tabelas Dimensão  
_São as "pontas" da estrela. Elas descrevem o contexto dos fatos._  
_Contêm os atributos descritivos e textuais usados para filtrar, agrupar e rotular os dados._  

In [0]:
# Criação Tabela dimensão Jogo
dim_jogo = df_tabuleiro.select(
    col("ID").alias("id_jogo"),
    col("Name").alias("nome_jogo"),
    col("Year Published").alias("ano_publicacao"),
).distinct()

# Adicionar chave substituta
dim_jogo = dim_jogo.withColumn("jogo_sk", monotonically_increasing_id())

# Visualiza amostra da tabela
print("Dimensão Jogo:")
display(dim_jogo.limit(5))

In [0]:
# Criação Tabela dimensão Jogabilidade
dim_jogabilidade = df_tabuleiro.select(
    col("ID").alias("id_jogo"),
    col("Min Players").alias("min_jogadores"),
    col("Max Players").alias("max_jogadores"),
    col("Play Time").alias("tempo_jogo"),
    col("Min Age").alias("idade_minima")
).distinct()

# Adicionar chave substituta
dim_jogabilidade = dim_jogabilidade.withColumn("jogabilidade_sk", monotonically_increasing_id())

# Visualiza amostra da tabela
print("Dimensão Jogabilidade:")
display(dim_jogabilidade.limit(5))

In [0]:
# Criação Tabela dimensão mecanicas
dim_mecanicas = df_tabuleiro.select(
    col("ID").alias("id_jogo"),
    explode(split(col("Mechanics"), ", ")).alias("mecanica_jogo")
).distinct()

# Adicionar chave substituta
dim_mecanicas = dim_mecanicas.withColumn("mecanica_sk", monotonically_increasing_id())

# Visualiza amostra da tabela
print("Dimensão Mecânicas:")
display(dim_mecanicas.limit(5))

In [0]:
# Criação Tabela dimensão Domínios
dim_dominios = df_tabuleiro.select(
    col("ID").alias("id_jogo"),
    explode(split(col("Domains"), ", ")).alias("dominio_jogo")
).distinct()

# Adicionar chave substituta
dim_dominios = dim_dominios.withColumn("dominio_sk", monotonically_increasing_id())

# Visualiza amostra da tabela
print("Dimensão Domínios:")
display(dim_dominios.limit(5))

#### Tabelas Fato  
_É o "coração" do modelo. Representa o evento ou processo de negócio que se deseja analisar._  
_Contém as medidas ou métricas numéricas._  
_É composta principalmente por chaves estrangeiras (FK) que se conectam às tabelas de dimensão e pelas medidas em si._

In [0]:
# Criação Tabela Fato Avaliacao
fato_avaliacao = df_tabuleiro.select(
    col("ID").alias("id_jogo"),
    col("Users Rated").alias("usuarios_avaliaram"),
    col("Rating Average").alias("media_avaliacao"),
    col("BGG Rank").alias("classificacao_bgg"),
    col("Complexity Average").alias("media_complexidade"),
    col("Owned Users").alias("usuarios_possuem")
)

# Visualiza amostra da tabela
print("Tabela Fato Avaliação:")
display(fato_avaliacao.limit(5))

#### Salvar as tabelas Delta no Databricks  
_As Tabelas Delta são tabelas que utilizam o formato Delta Lake._  
_Delta Lake é uma camada de armazenamento aberta que traz confiabilidade de transações ACID (Atomicidade, Consistência, Isolamento e Durabilidade) para data lakes._

In [0]:
# # No início do seu notebook, antes de criar as novas tabelas:
# print("INICIANDO LIMPEZA DO MODELO ESTRELA")
# print("-" * 40)

# # Limpar tabelas antigas
# tabelas_para_excluir = [
#     "tabuleiro.dim_jogo",
#     "tabuleiro.dim_jogabilidade",
#     "tabuleiro.dim_mecanicas", 
#     "tabuleiro.dim_dominios",
#     "tabuleiro.fato_avaliacao"
# ]

# for tabela in tabelas_para_excluir:
#     spark.sql(f"DROP TABLE IF EXISTS {tabela}")
#     print(f"★ {tabela} >> apagado")

# print("\n Ambiente sem tabelas.")

In [0]:
# # Salvar todos os dataframes do modelo estrela como tabelas Delta (recomendado para Databricks)
# dim_jogo.write.mode("overwrite").saveAsTable("tabuleiro.dim_jogo")
# dim_jogabilidade.write.mode("overwrite").saveAsTable("tabuleiro.dim_jogabilidade")
# dim_mecanicas.write.mode("overwrite").saveAsTable("tabuleiro.dim_mecanicas")
# dim_dominios.write.mode("overwrite").saveAsTable("tabuleiro.dim_dominios")
# fato_avaliacao.write.mode("overwrite").saveAsTable("tabuleiro.fato_avaliacao")

# # Verificar tabelas criadas
# print("\n Tabelas criadas no schema 'tabuleiro':")
# display(spark.sql("SHOW TABLES IN tabuleiro"))

In [0]:
# Criação do modelo estrela no Databricks

print("INICIANDO CRIAÇÃO DO MODELO ESTRELA")
print("-" * 50)

# Lista de tabelas do modelo estrela
tabelas = {
    "tabuleiro.dim_jogo": dim_jogo,
    "tabuleiro.dim_jogabilidade": dim_jogabilidade,
    "tabuleiro.dim_mecanicas": dim_mecanicas,
    "tabuleiro.dim_dominios": dim_dominios,
    "tabuleiro.fato_avaliacao": fato_avaliacao
}

# Apagar e recriar cada tabela
for nome, df in tabelas.items():
    # Apagar tabela Delta, caso exista no esquema 'tabuleiro'
    spark.sql(f"DROP TABLE IF EXISTS {nome}")
    
    # Salvar os dataframe como tabelas Delta no esquema 'tabuleiro'
    df.write.mode("overwrite").saveAsTable(nome)
    print(f"★ {nome} >>> criado")

# Mostrar tabelas criadas
print("\nTabelas criadas no schema 'tabuleiro':")
display(spark.sql("SHOW TABLES IN tabuleiro"))

In [0]:
# Adiciona comentários dos atributos das tabelas do modelo estrela

# --- Comentários para dim_jogo
comentarios_dim_jogo = {
    "jogo_sk": "Chave substituta artificial para a dimensão jogo",
    "id_jogo": "ID original do jogo no dataset BoardGameGeek (BGG)",
    "nome_jogo": "Nome completo e oficial do jogo de tabuleiro",
    "ano_publicacao": "Ano de publicação original do jogo"
}

for coluna, comentario in comentarios_dim_jogo.items():
    spark.sql(f"""
    ALTER TABLE tabuleiro.dim_jogo 
    ALTER COLUMN {coluna} COMMENT '{comentario}'
    """)

print("Comentários adicionados para dim_jogo")

# --- Comentários para dim_jogabilidade
comentarios_dim_jogabilidade = {
    "jogabilidade_sk": "Chave substituta artificial para a dimensão jogabilidade",
    "id_jogo": "ID original do jogo no dataset BoardGameGeek (BGG)",
    "min_jogadores": "Número mínimo de jogadores necessários",
    "max_jogadores": "Número máximo de jogadores suportados", 
    "tempo_jogo": "Duração média da partida em minutos",
    "idade_minima": "Idade mínima recomendada para jogar"
}

for coluna, comentario in comentarios_dim_jogabilidade.items():
    spark.sql(f"""
    ALTER TABLE tabuleiro.dim_jogabilidade 
    ALTER COLUMN {coluna} COMMENT '{comentario}'
    """)

print("Comentários adicionados para dim_jogabilidade")

# --- Comentários para dim_mecanicas
comentarios_dim_mecanicas = {
    "mecanica_sk": "Chave substituta artificial para a dimensão mecânicas",
    "id_jogo": "ID original do jogo no dataset BoardGameGeek (BGG)",
    "mecanica_jogo": "Regras e dinâmica de jogabilidade. Como o jogo funciona (regras/dinâmica)."
}

for coluna, comentario in comentarios_dim_mecanicas.items():
    spark.sql(f"""
    ALTER TABLE tabuleiro.dim_mecanicas 
    ALTER COLUMN {coluna} COMMENT '{comentario}'
    """)

print("Comentários adicionados para dim_mecanicas")

# --- Comentários para dim_dominios
comentarios_dim_dominios = {
    "dominio_sk": "Chave substituta artificial para a dimensão domínios", 
    "id_jogo": "ID original do jogo no dataset BoardGameGeek (BGG)",
    "dominio_jogo": "Categorias temáticas ou gêneros dos jogos. O que o jogo é (categoria/tema)."
}

for coluna, comentario in comentarios_dim_dominios.items():
    spark.sql(f"""
    ALTER TABLE tabuleiro.dim_dominios 
    ALTER COLUMN {coluna} COMMENT '{comentario}'
    """)

print("Comentários adicionados para dim_dominios")

# --- Comentários para fato_avaliacao
comentarios_fato_avaliacao = {
    "id_jogo": "ID original do jogo no dataset BoardGameGeek (BGG)",
    "usuarios_avaliaram": "Quantidade de usuários que avaliaram o jogo",
    "media_avaliacao": "Nota média recebida",
    "classificacao_bgg": "Posição no ranking da BoardGameGeek",
    "media_complexidade": "Nível médio de dificuldade/complexidade",
    "usuarios_possuem": "Quantidade de usuários que possuem o jogo"
}

for coluna, comentario in comentarios_fato_avaliacao.items():
    spark.sql(f"""
    ALTER TABLE tabuleiro.fato_avaliacao 
    ALTER COLUMN {coluna} COMMENT '{comentario}'
    """)

print("Comentários adicionados para fato_avaliacao")

In [0]:
# Adiciona etiquetas (tags) dos atributos das tabelas do modelo estrela

# --- Etiquetas para dim_jogo
etiquetas_dim_jogo = {
    "id_jogo": "identificador",
    "nome_jogo": "descricao",
    "ano_publicacao": "temporal",
    "jogo_sk": "chave_substituta"
}

for coluna, etiqueta in etiquetas_dim_jogo.items():
    spark.sql(f"""
    ALTER TABLE tabuleiro.dim_jogo 
    ALTER COLUMN {coluna} SET TAGS ('{etiqueta}')
    """)

print("Etiquetas adicionadas para dim_jogo")

# --- Etiquetas para dim_jogabilidade
etiquetas_dim_jogabilidade = {
    "jogabilidade_sk": "chave_substituta",
    "id_jogo": "identificador",
    "min_jogadores": "jogabilidade_min",
    "max_jogadores": "jogabilidade_max",
    "tempo_jogo": "duracao",
    "idade_minima": "faixa_etaria"
}

for coluna, etiqueta in etiquetas_dim_jogabilidade.items():
    spark.sql(f"""
    ALTER TABLE tabuleiro.dim_jogabilidade 
    ALTER COLUMN {coluna} SET TAGS ('{etiqueta}')
    """)

print("Etiquetas adicionadas para dim_jogabilidade")

# --- Etiquetas para dim_mecanicas
etiquetas_dim_mecanicas = {
    "mecanica_sk": "chave_substituta",
    "id_jogo": "identificador",
    "mecanica_jogo": "dinamica"
}

for coluna, etiqueta in etiquetas_dim_mecanicas.items():
    spark.sql(f"""
    ALTER TABLE tabuleiro.dim_mecanicas 
    ALTER COLUMN {coluna} SET TAGS ('{etiqueta}')
    """)

print("Etiquetas adicionadas para dim_mecanicas")

# --- Etiquetas para dim_dominios
etiquetas_dim_dominios = {
    "dominio_sk": "chave_substituta",
    "id_jogo": "identificador",
    "dominio_jogo": "categoria"
}

for coluna, etiqueta in etiquetas_dim_dominios.items():
    spark.sql(f"""
    ALTER TABLE tabuleiro.dim_dominios 
    ALTER COLUMN {coluna} SET TAGS ('{etiqueta}')
    """)

print("Etiquetas adicionadas para dim_dominios")

# --- Etiquetas para fato_avaliacao
etiquetas_fato_avaliacao = {
    "id_jogo": "identificador",
    "usuarios_avaliaram": "engajamento",
    "media_avaliacao": "desempenho",
    "classificacao_bgg": "ranking",
    "media_complexidade": "nivel",
    "usuarios_possuem": "popularidade"
}

for coluna, etiqueta in etiquetas_fato_avaliacao.items():
    spark.sql(f"""
    ALTER TABLE tabuleiro.fato_avaliacao 
    ALTER COLUMN {coluna} SET TAGS ('{etiqueta}')
    """)

print("Etiquetas adicionadas para fato_avaliacao")

#### Gráfico dos insight

In [0]:
# 01. Top 10 Jogos Melhor Avaliados

# Monta a query
query = """
SELECT 
    j.nome_jogo as Nome_Jogo,
    j.ano_publicacao as Ano_Publicacao,
    f.media_avaliacao as Avaliacao_Media,
    f.usuarios_avaliaram as Total_que_Avaliaram,
    f.classificacao_bgg as Ranking_BGG
FROM tabuleiro.fato_avaliacao f
JOIN tabuleiro.dim_jogo j ON f.id_jogo = j.id_jogo
WHERE f.usuarios_avaliaram > 1000
ORDER BY f.media_avaliacao DESC
LIMIT 10
"""

# Executa a query e carrega em um DataFrame Spark
df_spark = spark.sql(query)
df = df_spark.toPandas()

plt.figure(figsize=(14,8))
sns.barplot(data=df, x="Nome_Jogo", y="Total_que_Avaliaram", palette="viridis", hue="Nome_Jogo", legend=False)

# Adiciona os rótulos acima das barras
for i, row in df.iterrows():
    plt.text(i, row["Total_que_Avaliaram"] + 50, 
             f"{row['Total_que_Avaliaram']} (nota {row['Avaliacao_Media']:.2f})", 
             ha='center', va='bottom', fontsize=9)

plt.xticks(rotation=90, ha="right", fontsize=9)
plt.yticks(fontsize=9)
plt.title("Top 10: Jogos melhores avaliados", fontsize=12, weight="bold")
plt.ylabel("Total de Usuários que Avaliaram", fontsize=10)
plt.xlabel("Nome do Jogo", fontsize=10)

# Espaçamento extra no topo
plt.ylim(0, df["Total_que_Avaliaram"].max() * 1.15)

# Melhorias de borda e linhas internas
ax = plt.gca()
# Borda externa mais grossa
for spine in ax.spines.values():
    spine.set_linewidth(0.9)
    spine.set_color("black")

# Linhas horizontais internas
ax.yaxis.grid(True, linestyle="--", linewidth=0.7, alpha=0.7)

# Formatar eixo y em múltiplos de 10k
ax.yaxis.set_major_locator(MultipleLocator(10000))
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: f"{int(y/1000)}k"))

plt.tight_layout()
plt.show()

In [0]:
# 02. Média de Avaliação por Nível de Complexidade

# Monta a query
query = """
SELECT 
    CASE 
        WHEN f.media_complexidade < 2 THEN 'Iniciante (0-2)'
        WHEN f.media_complexidade < 3 THEN 'Intermediário (2-3)'
        WHEN f.media_complexidade < 4 THEN 'Avançado (3-4)'
        ELSE 'Expert (4-5)'
    END as Nivel_Complexidade,
    ROUND(AVG(f.media_avaliacao), 2) as Media_Avaliacao,
    COUNT(*) as Total_Jogos
FROM tabuleiro.fato_avaliacao f
GROUP BY Nivel_Complexidade 
ORDER BY Media_Avaliacao ASC
"""

# Executa a query e carrega em um DataFrame Spark
df_spark = spark.sql(query)
df = df_spark.toPandas()

plt.figure(figsize=(10,6))
sns.barplot(data=df, y="Nivel_Complexidade", x="Media_Avaliacao", palette="mako", legend=False, hue="Nivel_Complexidade")

# Adiciona rótulos com média e total de jogos
for i, row in df.iterrows():
    plt.text(row["Media_Avaliacao"] + 0.05, i, 
             f"{row['Media_Avaliacao']:.2f} ★ |  {row['Total_Jogos']} jogos", 
             va='center', fontsize=8)

plt.title("Média de Avaliação por Nível de Complexidade", fontsize=13, weight="bold")
plt.xlabel("Média de Avaliação", fontsize=10)
plt.ylabel("Nível de Complexidade", fontsize=10)

# Ajustes visuais
plt.grid(axis="x", linestyle="--", alpha=0.6)
plt.yticks(fontsize=9)
plt.xticks(fontsize=9)
plt.xlim(0, df["Media_Avaliacao"].max() * 1.2)  # espaço extra no topo

# Melhorias de borda e linhas internas
ax = plt.gca()
# Borda externa mais grossa
for spine in ax.spines.values():
    spine.set_linewidth(0.9)
    spine.set_color("black")

# Linhas verticais internas
#ax.yaxis.grid(True, linestyle="--", linewidth=0.7, alpha=0.7)  # linhas horizontais
ax.xaxis.grid(True, linestyle="--", linewidth=0.7, alpha=0.5)  # linhas verticais

plt.tight_layout()
plt.show()

In [0]:
# 03. Mecânicas mais populares

# Monta a query
query = """
SELECT 
    dm.mecanica_jogo AS Mecanica_Jogo, 
    SUM(fa.usuarios_possuem) AS Popularidade
FROM tabuleiro.fato_avaliacao fa
JOIN tabuleiro.Dim_Mecanicas dm 
    ON fa.id_jogo = dm.id_jogo
GROUP BY dm.mecanica_jogo
ORDER BY Popularidade DESC
LIMIT 10;
"""

# Executa a query e carrega em um DataFrame Spark
df_spark = spark.sql(query)
df = df_spark.toPandas().sort_values("Popularidade", ascending=False)

plt.figure(figsize=(10,6))
ax = sns.barplot(data=df, y="Mecanica_Jogo", x="Popularidade", palette="flare", legend=False, hue="Mecanica_Jogo")

# Adiciona rótulos com popularidade formatada
for i, row in df.iterrows():
    plt.text(
        row["Popularidade"] + (row["Popularidade"] * 0.02),
        i,
        f"★ {row['Popularidade']:,} usuários",
        va="center",
        fontsize=8
    )

# Título e eixos
plt.title("Top 10: Mecânicas de jogos mais populares", fontsize=13, weight="bold")
plt.xlabel("Número de Usuários Proprietários", fontsize=10)
plt.ylabel("Mecânica de Jogo", fontsize=10)

# Ajustes visuais
plt.grid(axis="x", linestyle="--", alpha=0.6)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.xlim(0, df["Popularidade"].max() * 1.3) # Espaço extra para rótulos

# Formatar eixo X em milhares (k)
ax.xaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{int(x/1000)}k" if x >= 1000 else f"{int(x)}"))

# Melhorias de borda e linhas internas
ax = plt.gca()
# Borda externa mais grossa
for spine in ax.spines.values():
    spine.set_linewidth(0.9)
    spine.set_color("black")

# Linhas verticais internas
#ax.yaxis.grid(True, linestyle="--", linewidth=0.7, alpha=0.7)  # linhas horizontais
ax.xaxis.grid(True, linestyle="--", linewidth=0.7, alpha=0.5)  # linhas verticais

plt.tight_layout()
plt.show()

In [0]:
# 04. Complexidade média e tempo de jogo

# Monta a query
query = """
WITH jogos AS (
    SELECT 
        favalia.id_jogo, djogab.tempo_jogo,
        CASE 
            WHEN favalia.media_complexidade < 2 THEN 'Baixa (0-2)'
            WHEN favalia.media_complexidade < 3 THEN 'Média (2-3)'
            WHEN favalia.media_complexidade < 4 THEN 'Alta (3-4)'
            ELSE 'Muito Alta (4-5)'
        END AS Faixa_Complexidade
    FROM tabuleiro.fato_avaliacao favalia
    JOIN tabuleiro.dim_jogo djogo 
        ON favalia.id_jogo = djogo.id_jogo
    JOIN tabuleiro.dim_jogabilidade djogab 
        ON favalia.id_jogo = djogab.id_jogo
) 
SELECT 
    Faixa_Complexidade,
    ROUND(AVG(tempo_jogo), 2) AS Tempo_Medio_Jogo,
    COUNT(DISTINCT id_jogo) AS Total_Jogos
FROM jogos
GROUP BY Faixa_Complexidade
ORDER BY Tempo_Medio_Jogo ASC
"""

# Executa a query e carrega em um DataFrame Spark
df_spark = spark.sql(query)
df = df_spark.toPandas()

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,6))
sns.barplot(data=df, y="Faixa_Complexidade", x="Tempo_Medio_Jogo", palette="flare", legend=False, hue="Faixa_Complexidade")

# Adiciona rótulos com tempo médio e total de jogos
for i, row in df.iterrows():
    plt.text(row["Tempo_Medio_Jogo"] + 1, i, 
             f"★ {row['Tempo_Medio_Jogo']} min | {row['Total_Jogos']} jogos", 
             va='center', fontsize=8)

plt.title("Relação entre Complexidade Média e Tempo de Jogo", fontsize=13, weight="bold")
plt.xlabel("Tempo Médio de Jogo (minutos)", fontsize=10)
plt.ylabel("Faixa de Complexidade", fontsize=10)

# Ajustes visuais
plt.grid(axis="x", linestyle="--", alpha=0.6)
plt.xticks(fontsize=9)
plt.yticks(fontsize=9)
plt.xlim(0, df["Tempo_Medio_Jogo"].max() * 1.3)  # espaço extra para rótulos

# Melhorias de borda e linhas internas
ax = plt.gca()
# Borda externa mais grossa
for spine in ax.spines.values():
    spine.set_linewidth(0.9)
    spine.set_color("black")

# Linhas verticais internas
#ax.yaxis.grid(True, linestyle="--", linewidth=0.7, alpha=0.7)  # linhas horizontais
ax.xaxis.grid(True, linestyle="--", linewidth=0.7, alpha=0.5)  # linhas verticais

plt.tight_layout()
plt.show()

In [0]:
# 05. Comparação da Média de Avaliação por Faixa Etária

# Monta a query
query = """
SELECT 
    CASE 
        WHEN djg.idade_minima BETWEEN 0 AND 5 THEN 'Infantil (0-5)'
        WHEN djg.idade_minima BETWEEN 6 AND 12 THEN 'Pré-adolescente (6-12)'
        WHEN djg.idade_minima BETWEEN 13 AND 17 THEN 'Adolescente (13-17)'
        ELSE 'Adulto (18-21)'
    END AS Faixa_Etaria_Jogador,
    ROUND(AVG(fa.media_avaliacao), 2) AS Media_Avaliacao_Jogo,
    COUNT(DISTINCT fa.id_jogo) AS Total_Jogos
FROM tabuleiro.fato_avaliacao fa
JOIN tabuleiro.dim_jogabilidade djg 
    ON fa.id_jogo = djg.id_jogo
GROUP BY Faixa_Etaria_Jogador
ORDER BY Media_Avaliacao_Jogo ASC
"""

# Executa a query e carrega em um DataFrame Spark
df_spark = spark.sql(query)
df = df_spark.toPandas()

plt.figure(figsize=(10,6))
sns.barplot(data=df, y="Faixa_Etaria_Jogador", x="Media_Avaliacao_Jogo", palette="crest", legend=False, hue="Faixa_Etaria_Jogador")

# Adiciona rótulos com média e total de jogos
for i, row in df.iterrows():
    plt.text(row["Media_Avaliacao_Jogo"] + 0.05, i, 
             f"★ {row['Media_Avaliacao_Jogo']:.2f} | {row['Total_Jogos']} jogos", 
             va='center', fontsize=8)

plt.title("Comparação da Média de Avaliação por Faixa Etária (0 a 21)", fontsize=13, weight="bold")
plt.xlabel("Média de Avaliação do Jogo", fontsize=10)
plt.ylabel("Faixa Etária do Jogador", fontsize=10)

# Ajustes visuais
plt.grid(axis="x", linestyle="--", alpha=0.6)
plt.xticks(fontsize=9)
plt.yticks(fontsize=9)
plt.xlim(0, df["Media_Avaliacao_Jogo"].max() * 1.2)  # espaço extra no topo

# Melhorias de borda e linhas internas
ax = plt.gca()
# Borda externa mais grossa
for spine in ax.spines.values():
    spine.set_linewidth(0.9)
    spine.set_color("black")

# Linhas verticais internas
#ax.yaxis.grid(True, linestyle="--", linewidth=0.7, alpha=0.7)  # linhas horizontais
ax.xaxis.grid(True, linestyle="--", linewidth=0.7, alpha=0.5)  # linhas verticais

plt.tight_layout()
plt.show()

In [0]:
# 06. Domínios mais populares com melhores médias de avaliação

# Monta a query
query = """
SELECT 
    dd.dominio_jogo AS Categoria,
    SUM(fb.usuarios_possuem) AS Popularidade,
    AVG(fb.media_avaliacao) AS Media_Avaliacao
FROM tabuleiro.fato_avaliacao fb
JOIN tabuleiro.dim_dominios dd 
    ON fb.id_jogo = dd.id_jogo
WHERE fb.media_avaliacao IS NOT NULL
GROUP BY dd.dominio_jogo
ORDER BY Media_Avaliacao DESC, Popularidade DESC
"""

# Executa a query e carrega em um DataFrame Spark
df_spark = spark.sql(query)

# Converter para Pandas
df = df_spark.toPandas()

# Plotar gráfico com Seaborn
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="whitegrid")
plt.figure(figsize=(10,6))

# Gráfico de barras horizontal ordenado pela média de avaliação
ax = sns.barplot(
    data=df,
    x="Media_Avaliacao",
    y="Categoria",
    palette="viridis",
    legend=False, 
    hue= "Categoria"
)

# Adicionar rótulos com média e popularidade
for i, (val, pop) in enumerate(zip(df["Media_Avaliacao"], df["Popularidade"])):
    ax.text(val + 0.05, i, f"★ {val:.2f} | Pop: {pop}", 
            va="center", fontsize=8, color="black")

# Título e rótulos
plt.title("Domínios mais populares vs médias de avaliação", fontsize=16, weight="bold")
plt.xlabel("Média de Avaliação", fontsize=11)
plt.ylabel("Categoria", fontsize=11)

plt.xticks(fontsize=9)
plt.yticks(fontsize=9)
plt.xlim(0, df["Media_Avaliacao"].max() * 1.2)  # espaço extra no topo

# Borda externa mais grossa
for spine in ax.spines.values():
    spine.set_linewidth(0.9)
    spine.set_color("black")

# Linhas verticais internas
#ax.yaxis.grid(True, linestyle="--", linewidth=0.7, alpha=0.7)  # linhas horizontais
ax.xaxis.grid(True, linestyle="--", linewidth=0.7, alpha=0.5)  # linhas verticais

plt.tight_layout()
plt.show()

In [0]:
df = spark.sql("""
SELECT 
    CASE 
        WHEN j.tempo_jogo <= 30 THEN 'Curto (≤30 min)'
        WHEN j.tempo_jogo BETWEEN 31 AND 90 THEN 'Médio (31-90 min)'
        ELSE 'Longo (>90 min)'
    END AS faixa_tempo,
    ROUND(AVG(f.media_avaliacao),2) AS media_avaliacao,
    COUNT(DISTINCT f.id_jogo) AS qtd_jogos
FROM tabuleiro.fato_avaliacao f
JOIN tabuleiro.dim_jogabilidade j ON f.id_jogo = j.id_jogo
GROUP BY faixa_tempo
ORDER BY media_avaliacao DESC
""")

pdf = df.toPandas()

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,6))
ax = sns.barplot(   # <<< captura o objeto Axes
    x="faixa_tempo", 
    y="media_avaliacao", 
    data=pdf, 
    palette="Blues_d"
)

# Adiciona rótulos com qtd_jogos
for i, row in pdf.iterrows():
    ax.text(
        i, row["media_avaliacao"] + 0.05, 
        f'{row["qtd_jogos"]} jogos', 
        ha='center', va='bottom', fontsize=9, color='black'
    )

ax.set_title("Tempo de Jogo vs. Avaliação Média", fontsize=14, weight="bold")
ax.set_xlabel("Faixa de Tempo de Jogo", fontsize=11)
ax.set_ylabel("Média de Avaliação", fontsize=11)

# 🔽 Ajuste do tamanho da fonte dos valores dos eixos
plt.xticks(fontsize=9)
plt.yticks(fontsize=9)

# Borda externa mais grossa
for spine in ax.spines.values():
    spine.set_linewidth(0.9)
    spine.set_color("black")

# Linhas horizontais internas
ax.yaxis.grid(True, linestyle="--", linewidth=0.7, alpha=0.7)

plt.tight_layout()
plt.show()
